In [6]:
!pip freeze

asttokens==2.0.8
backcall==0.2.0
certifi @ file:///C:/b/abs_4f5wo627a3/croots/recipe/certifi_1663615677642/work/certifi
charset-normalizer==2.1.1
colorama==0.4.5
debugpy==1.6.3
decorator==4.4.2
entrypoints==0.4
executing==1.1.0
idna==3.4
imageio==2.22.0
imageio-ffmpeg==0.4.7
ipyfilechooser==0.6.0
ipykernel==6.16.0
ipython==8.5.0
ipywidgets==8.0.2
jedi==0.18.1
jupyter-core==4.11.1
jupyter_client==7.3.5
jupyterlab-widgets==3.0.3
matplotlib-inline==0.1.6
mkl-fft==1.3.1
mkl-random @ file:///C:/ci/mkl_random_1626186184308/work
mkl-service==2.4.0
moviepy==1.0.3
nest-asyncio==1.5.5
numpy @ file:///C:/Windows/Temp/abs_2a1e1vbeag/croots/recipe/numpy_and_numpy_base_1659432712056/work
packaging==21.3
pandas==1.5.0
parso==0.8.3
pickleshare==0.7.5
Pillow==9.2.0
proglog==0.1.10
prompt-toolkit==3.0.31
psutil==5.9.2
pure-eval==0.2.2
Pygments==2.13.0
pyparsing==3.0.9
python-dateutil==2.8.2
pytz==2022.2.1
pywin32==304
pyzmq==24.0.1
requests==2.28.1
six @ file:///tmp/build/80754af9/six_1644875935023/work

In [8]:
!python ./pipline/train.py -dataset TSU -mode rgb -split_setting CS -model PDAN -train True -num_channel 512 -lr 0.0002 -kernelsize 2 -APtype map -epoch 1 -batch_size 1 -comp_info TSU_CS_RGB_PDAN -load_model ./pipline/dataset/PDAN_TSU_RGB

Random_SEED!!!: 0
PDAN
batch_size: 1
cuda_avail True
RGB mode ./pipline/RGB_i3d_16frames_64000_SSD
./pipline/data/smarthome_CS_51.json ./pipline/data/smarthome_CS_51.json ./pipline/RGB_i3d_16frames_64000_SSD 1 51
Random_SEED!!!: 0
Random_SEED!!!: 0
Random_SEED!!!: 0
Random_SEED!!!: 0
Random_SEED!!!: 0
Random_SEED!!!: 0
Random_SEED!!!: 0
Random_SEED!!!: 0
Random_SEED!!!: 0
Random_SEED!!!: 0
you are processing PDAN
loaded ./pipline/dataset/PDAN_TSU_RGB
pytorch_total_params 5804083
num_channel: 512 input_channnel: 1024 num_classes: 51
0.0002
Epoch 0/0
----------
train-map: tensor(44.9021)
val-map: tensor(32.0773)
tensor([42.4594, 48.5667, 41.7251, 42.3732, 40.6359, 43.3911, 39.2665, 28.5796,
        41.4261, 19.2768, 17.7311,  6.0897,  0.6160, 35.8210, 37.1473,  2.9251,
        22.4334, 52.2854,  2.6481, 70.7477, 27.3019, 45.6452, 52.7737,  2.8811,
         1.5986, 62.0351, 33.7371, 54.8382, 58.9922, 14.6661,  5.5195,  1.3654,
        39.3078,  2.3473, 42.4926, 86.0436, 39.7756, 15.0927, 


100%|##########| 536/536 [00:05<00:00, 93.36it/s] 

100%|##########| 536/536 [00:03<00:00, 163.29it/s]
C:\Users\jaspz\Documents\GitHub\ict3104-team10-2022\pipline\models.py:88: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.key_conv.weight, mode='fan_out')
C:\Users\jaspz\Documents\GitHub\ict3104-team10-2022\pipline\models.py:89: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.value_conv.weight, mode='fan_out')
C:\Users\jaspz\Documents\GitHub\ict3104-team10-2022\pipline\models.py:90: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.query_conv.weight, mode='fan_out')
C:\Users\jaspz\Documents\GitHub\ict3104-team10-2022\pipline\models.py:91: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(self.rel_t, 0, 1)
C:\Users\jaspz\anaconda3\envs\04Env\

# Installations & Util

In [1]:
! pip install ipyfilechooser ipywidgets pandas moviepy

## Code Hiding

In [2]:
import ipywidgets as widgets
from IPython.display import display, HTML
# ========== TOGGLE BTN ==========
javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """
    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)
    display(HTML(output))

def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new
    toggle_code(state)
    value.owner.description = button_descriptions[state]
    
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")
# ========== END TOGGLE BTN END ==========
display(button)

ToggleButton(value=False, description='Show code')

# Data Exploration

## File Upload

In [3]:
import ipywidgets as widgets
from IPython.display import display, HTML
from ipyfilechooser import FileChooser
import zipfile as zf
from ipywidgets import Button

# ========== FileChooser ==========
# Create and display a FileChooser widget
fc = FileChooser('/Users')

# Change defaults and reset the dialog
# fc.default_path = './'
# fc.reset()

# Restrict navigation to /Users
fc.sandbox_path = '/Users'

# Change hidden files
fc.show_hidden = False

# Switch to folder-only mode
fc.show_only_dirs = False

# Set multiple file filter patterns (uses https://docs.python.org/3/library/fnmatch.html)
# fc.filter_pattern = ['*.jpg', '*.png','*.txt']
# fc.filter_pattern = ['*.zip']
fc.filter_pattern = ['*.png']

# Change the title (use '' to hide)
fc.title = '<b>Select File (*zip)</b>'

# Sample callback function
def change_display_selected(chooser):
    print(fc.selected_filename, end='\r')

# Register callback function
fc.register_callback(change_display_selected)

def on_button_clicked(b):
    # FILE UPLOAD
    print(fc.selected)
    fn = fc.selected_filename.replace('.zip', '')
    files = zf.ZipFile(fc.selected_path+"\\"+fc.selected_filename, 'r')
    files.extractall('Data Folder\\'+fn)
    files.close()
    with output:
        print("Files saved.")
# ========== END FileChooser END ==========
# ========== CONFIRM BTN ==========
confirmBtn = widgets.Button(
    description='Confirm Upload',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Confirm',
)
output = widgets.Output()
confirmBtn.on_click(on_button_clicked)
# ========== CONFIRM BTN ==========
display(fc)
display(confirmBtn, output)

FileChooser(path='C:\Users', filename='', title='<b>Select File (*zip)</b>', show_hidden=False, select_desc='S…

Button(button_style='info', description='Confirm Upload', style=ButtonStyle(), tooltip='Confirm')

Output()

# Inference

In [ ]:
import cv2
import pandas as pd
from moviepy.editor import VideoFileClip

def output(frame):
    try:
        cv2.putText(frame, str(next(df)[1].sentence), position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3, cv2.LINE_AA)
    except StopIteration:
        pass
    # additional frame manipulation
    return frame


# Path to video
video_path = "video.mp4"

# Capture video
cap = cv2.VideoCapture(video_path)

# Get video height and width from captured video
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Set position for text with offset of text size
position = ((int((frame_width/2)-(268/2))), int(((frame_height/2)-(36/2))))

video = VideoFileClip(video_path)

# Path to caption
caption = "caption.csv"

df = pd.read_csv(caption).iterrows()

# Modifies the images of a clip by replacing the frame
out_video = video.fl_image(output)

# Write edited video to file
out_video.write_videofile("output.mp4", audio = True)

# Close video file clip
out_video.close()

# Training

## Select File for Training

In [ ]:
from IPython import display
from ipyfilechooser import FileChooser
from IPython.display import display
from IPython.display import IFrame
import ipywidgets as widgets
from IPython.display import Video

output2 = widgets.Output()
# ========== FileChooser ==========
# Create and display a FileChooser widget
fc2 = FileChooser('./Data Folder')
display(fc2)

# Restrict navigation to /Users
fc2.sandbox_path = './Data Folder'

# Change hidden files
fc2.show_hidden = False

# Switch to folder-only mode
fc2.show_only_dirs = False

# Change the title (use '' to hide)
fc2.title = '<b>Select File for Training</b>'

# Sample callback function
def change_display_selected(chooser):
    with output2:
#         print(fc2.selected_filename, end='\r')
        print(fc2.selected_path+"\\"+fc2.selected_filename, end='\r')
#     runVid(fc2.selected_path+"\\"+fc2.selected_filename)
#         Video(fc2.selected_path+"\\"+fc2.selected_filename)
    
# Register callback function
fc2.register_callback(change_display_selected)
# ========== END FileChooser END ==========
display(output2)

# Testing

Insert testing components